In [2]:
import pandas as pd 
import glob 
import numpy as np

# ML - scikit-learn and pytorch libraries
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

Loading in the flight state vector paths (using the accident for now as querying a sample of normal flights atm)

In [3]:
flight_state_vectors_paths = glob.glob("../../data/raw/accident_flight_states/*")

flight_state_vectors_paths

['../../data/raw/accident_flight_states/state_vectors_a52acc_1654088894_1654111587.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a255d9_1588773983_1588801421.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a7e3cb_1580225813_1580235289.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a88cbf_1616951147_1616955197.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a052d4_1727960352_1727965975.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a81344_1657884224_1657890629.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a6bac5_1697919371_1697934368.parquet',
 '../../data/raw/accident_flight_states/state_vectors_4ca9cb_1693357437_1693378258.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a0907b_1591456482_1591457426.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a18899_1675518416_1675524470.parquet',
 '../../data/raw/accident_flight_states/state_vectors_a449a1_169608957

In [17]:
# Process flight state vector data by:
# 1. Loading individual parquet files containing state vectors
# 2. Resampling each flight's data to 5 second intervals to standardize the sampling rate
# 3. Combining all resampled flight data into a single DataFrame

def resample_flight_state_data(df, interval='5s'):
    """
    Resample flight state vector data to a specified time interval.
    
    This function takes raw flight state data with irregular time intervals and resamples it
    to create evenly spaced observations. For each new time interval, it takes the first 
    available state vector if multiple observations exist within that interval.
    
    Args:
        df (pd.DataFrame): DataFrame containing flight state vector data. Must have a 'time' 
                          column with Unix timestamps.
        interval (str): Pandas time interval string specifying the desired sampling rate.
                       Default is '5s' for 5 seconds. See pandas documentation for other options.
        
    Returns:
        pd.DataFrame: A new DataFrame with state vectors resampled to the specified interval.
                     The time column is converted to datetime and all rows are sorted chronologically.
    """
    # Create a copy to avoid modifying the original data
    df_copy = df.copy()
    
    # Convert Unix timestamps to pandas datetime objects
    df_copy['time'] = pd.to_datetime(df_copy['time'], unit='s')
    
    # Ensure data is sorted chronologically
    df_copy = df_copy.sort_values('time')
    
    # Set time as index for resampling
    df_copy.set_index('time', inplace=True)
    
    # Resample data:
    # - Creates new rows at regular intervals specified by 'interval'
    # - Takes the first observation in each interval
    # - Resets index to make time a regular column again
    resampled_df = df_copy.resample(interval).first().reset_index()
    
    return resampled_df

# Initialize list to store resampled DataFrames
resampled_dfs = []

# Process each flight's state vector file with forward fill imputation
for path in flight_state_vectors_paths:
    # Load the raw state vector data
    df = pd.read_parquet(path)
    
    # Resample to 5-second intervals
    resampled_df = resample_flight_state_data(df)
    
    # Drop columns that are entirely NaN
    resampled_df = resampled_df.dropna(axis=1, how='all')

    resampled_df = resampled_df.sort_values('time')  # ensure order
    resampled_df.set_index('time', inplace=True)  # if you have a time index
    
    # interpolate missing values
    resampled_df[['lat', 'lon', 'velocity', 'heading', 'vertrate', 'geoaltitude', 'baroaltitude']] = resampled_df[['lat', 'lon', 'velocity', 'heading', 'vertrate', 'geoaltitude', 'baroaltitude']].interpolate(method='time')
    
    # unset index
    resampled_df.reset_index(drop=False, inplace=True)

    # Append only non-empty DataFrames to the list
    if not resampled_df.empty:
        resampled_dfs.append(resampled_df)

# Concatenate all imputed and resampled DataFrames
resampled_df = pd.concat(resampled_dfs, ignore_index=True)

# Display information about the final DataFrame
resampled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392310 entries, 0 to 392309
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype          
---  ------        --------------   -----          
 0   time          392310 non-null  datetime64[ns] 
 1   icao24        258056 non-null  string         
 2   lat           392305 non-null  double[pyarrow]
 3   lon           392305 non-null  double[pyarrow]
 4   velocity      392115 non-null  double[pyarrow]
 5   heading       392115 non-null  double[pyarrow]
 6   vertrate      392115 non-null  double[pyarrow]
 7   callsign      253998 non-null  string         
 8   onground      258056 non-null  bool[pyarrow]  
 9   spi           258056 non-null  bool[pyarrow]  
 10  squawk        118846 non-null  string         
 11  geoaltitude   392095 non-null  double[pyarrow]
 12  baroaltitude  392303 non-null  double[pyarrow]
dtypes: bool[pyarrow](2), datetime64[ns](1), double[pyarrow](7), string(3)
memory usage: 37.1 MB


In [20]:
# lets check missing values
resampled_df.isna().sum()


time                 0
icao24          134254
lat                  5
lon                  5
velocity           195
heading            195
vertrate           195
callsign        138312
onground        134254
spi             134254
squawk          273464
geoaltitude        215
baroaltitude         7
dtype: int64

In [21]:
# now lets drop rows with missing - lat, lon, velocity, heading, vertrate, geoaltitude, baroaltitude
resampled_df = resampled_df.dropna(subset=['lat', 'lon', 'velocity', 'heading', 'vertrate', 'geoaltitude', 'baroaltitude'])

# lets check missing values again
resampled_df.isna().sum()



time                 0
icao24          134251
lat                  0
lon                  0
velocity             0
heading              0
vertrate             0
callsign        138297
onground        134251
spi             134251
squawk          273436
geoaltitude          0
baroaltitude         0
dtype: int64

Now we can create the input and output sequences, creating this sliding window of 17 steps and predicting the next 1 step

we first define `create_input_output_sequences` function which will 
- Each input window (X) contains 17 consecutive time steps of flight data
- Each output (y) contains the next 1 time step after the input window
- The function slides this window through the entire flight, creating many (X,y) pairs for training


In [22]:
def create_input_output_sequences(
    df_flight, 
    input_len=17, 
    pred_len=1, 
    feature_cols=None, 
    target_cols=None
):
    """
    Create pairs of (X, y) from a single flight's data for one-step-ahead prediction:
      - X: the first 'input_len' time steps
      - y: the next 'pred_len' time steps (1 in this case).
    
    Args:
        df_flight (pd.DataFrame): Time-sorted data for a single flight.
        input_len (int): Number of historical time steps used as input.
        pred_len (int): Number of time steps to predict. For single-step, it's 1.
        feature_cols (list): Columns to use as inputs (X).
        target_cols (list): Columns to predict (y).
        
    Returns:
        X_array (np.ndarray): shape (num_samples, input_len, num_features)
        y_array (np.ndarray): shape (num_samples, pred_len, num_targets)
          - For single-step forecast, pred_len=1, so shape = (num_samples, 1, num_targets).
    """
    # If not specified, assume all numeric columns except flight_id/time are features
    if feature_cols is None:
        feature_cols = [c for c in df_flight.columns 
                        if c not in ['flight_id', 'time']]
    if target_cols is None:
        # Predict same as feature columns
        target_cols = feature_cols
    
    # Convert to numpy
    feature_values = df_flight[feature_cols].values
    target_values = df_flight[target_cols].values
    
    # We'll collect sequences here
    X_list, y_list = [], []
    
    n = len(df_flight)
    # We need a total of input_len + pred_len steps for each sample
    # e.g. 17 + 1 = 18
    seq_len = input_len + pred_len
    
    for start_idx in range(n - seq_len + 1):
        # X = [start_idx : start_idx+17]
        X_seq = feature_values[start_idx : start_idx + input_len]
        # y = [start_idx+17 : start_idx+17+1]
        y_seq = target_values[start_idx + input_len : start_idx + input_len + pred_len]
        
        X_list.append(X_seq)
        y_list.append(y_seq)
        
    X_array = np.array(X_list)  # shape: (num_samples, 17, num_features)
    y_array = np.array(y_list)  # shape: (num_samples, 1, num_targets)
    
    return X_array, y_array


In [ ]:
# the lists to store the input and output sequences
all_X = []
all_y = []

for flight_id, df_group in resampled_df.groupby('icao24'):
    # Sort by time
    df_group = df_group.sort_values('time').reset_index(drop=True)
    
    # Example feature columns
    feature_cols = ['lon','lat','heading','velocity','vertrate', 'heading', 'geoaltitude']
    # Example target columns: let's assume we want to predict the same set 
    target_cols = ['lon','lat','heading','velocity','vertrate', 'heading', 'geoaltitude']
    
    X_flight, y_flight = create_input_output_sequences(
        df_group, 
        input_len=17, 
        pred_len=1, 
        feature_cols=feature_cols, 
        target_cols=target_cols
    )
    
    all_X.append(X_flight)
    all_y.append(y_flight)

# Concatenate across all flights
X_final = np.concatenate(all_X, axis=0)  # shape: (total_samples, 17, num_features)
y_final = np.concatenate(all_y, axis=0)  # shape: (total_samples, 1, num_targets)

print("X_final shape:", X_final.shape)
print("y_final shape:", y_final.shape)


X_final shape: (254614, 17, 7)
y_final shape: (254614, 1, 7)


In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.2, shuffle=True, random_state=42 # use a random state to ensure reproducibility
)


In [25]:
# show examples of the data and its shape
print("X_train shape:", X_train.shape)
print("X_train dtype:", X_train.dtype)
print("y_train shape:", y_train.shape) 
print("y_train dtype:", y_train.dtype)
print("X_test shape:", X_test.shape)
print("X_test dtype:", X_test.dtype)
print("y_test shape:", y_test.shape)
print("y_test dtype:", y_test.dtype)


X_train shape: (203691, 17, 7)
X_train dtype: object
y_train shape: (203691, 1, 7)
y_train dtype: object
X_test shape: (50923, 17, 7)
X_test dtype: object
y_test shape: (50923, 1, 7)
y_test dtype: object


 
Now we can get into training the model, first we define the model using torch neural network modules

**Model Overview**  

- **Input Projection**: A linear layer projects the input features to a higher-dimensional space (`d_model=64`).
- **Positional Encoding**: Since Transformers do not inherently understand sequence order, we add positional encodings.
- **Transformer Encoder**: A multi-layer Transformer encoder processes the sequence using self-attention and feedforward layers.
- **Final Prediction**: We extract the final timestep’s representation from the Transformer output and pass it through a fully connected layer to predict the next state.

**Code Structure**  

- `TransformerPredictor`: Defines the Transformer-based model.
- `PositionalEncoding`: Implements sinusoidal positional encoding to provide sequence information.

The following cell contains the full implementation of the model.


In [26]:

class TransformerPredictor(nn.Module):
    def __init__(self, input_dim=5, d_model=64, nhead=4, num_layers=2, 
                 dim_feedforward=128, dropout=0.1, target_dim=5):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout=dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True  # If using PyTorch 1.10+ for batch_first in the encoder
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # We'll predict next state => shape (batch_size, target_dim)
        self.fc_out = nn.Linear(d_model, target_dim)
        
    def forward(self, x):
        """
        x shape: (batch_size, seq_len=17, input_dim=5)
        Returns: (batch_size, target_dim=5)
        """
        # Project input to d_model
        x = self.input_proj(x)  # shape: (batch_size, seq_len, d_model)
        
        # Add positional encoding
        x = self.pos_encoder(x)  # shape: (batch_size, seq_len, d_model)
        
        # Pass through Transformer encoder
        encoded = self.transformer_encoder(x)  # shape: (batch_size, seq_len, d_model)
        
        # We only want the final time-step's representation (seq_len-1 = 16)
        last_step = encoded[:, -1, :]  # shape: (batch_size, d_model)
        
        # Predict next state
        out = self.fc_out(last_step)  # (batch_size, target_dim)
        return out

# Example positional encoding implementation
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape: (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x shape: (batch_size, seq_len, d_model)
        """
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        return self.dropout(x)


In [44]:
# ensuring we are using the GPU
print(f"Is CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Is CUDA available: True
Current device: 0
Device name: NVIDIA GeForce RTX 3070


In [32]:
# Convert numpy arrays to proper numeric type before torch conversion
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

X_train_t = torch.from_numpy(X_train).float().to(device)  # (num_samples, 17, 7)
y_train_t = torch.from_numpy(y_train).float().to(device)  # (num_samples, 1, 7)
y_train_t = y_train_t.squeeze(1)               # shape: (num_samples, 7), if single-step

train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model = TransformerPredictor(input_dim=7, target_dim=7)
criterion = nn.MSELoss()  # or L1Loss, depends on your preference
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model.train()
for epoch in range(10):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)  # shape (batch_size, 5)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

Epoch 0, Loss: 3316818.0741
Epoch 1, Loss: 178314.2192
Epoch 2, Loss: 43906.9187
Epoch 3, Loss: 41371.2203
Epoch 4, Loss: 36641.0136
Epoch 5, Loss: 34304.3319
Epoch 6, Loss: 31459.4843
Epoch 7, Loss: 32426.3013
Epoch 8, Loss: 34311.9144
Epoch 9, Loss: 32364.1478


The drop in loss is promising ater only 10 epochs -> transformers need a lot of epochs to train

In [ ]:
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

model.eval()
X_test_t = torch.from_numpy(X_test).float().to(device)
y_test_t = torch.from_numpy(y_test).float().squeeze(1).to(device)

with torch.no_grad():
    preds = model(X_test_t)
    test_loss = criterion(preds, y_test_t)
    
print("Test Loss:", test_loss.item())

Test Loss: 21849.83203125


Now we can make predictions

In [ ]:
def pretty_print_state(state, columns):
    """
    Print the state in a human-readable format.

    Args:
        state (np.ndarray): 1D array of flight state values.
        columns (list of str): Names of the flight state dimensions.
    """
    for name, value in zip(columns, state):
        print(f"{name:15s}: {value:.2f}")

def print_prediction(input_seq, prediction, input_columns=None, pred_columns=None):
    """
    Print the last timestep of the input sequence and the predicted next state.

    Args:
        input_seq (np.ndarray): 2D array of shape (seq_len, num_features) for the input sequence.
        prediction (np.ndarray): 1D array of predicted flight state values.
        input_columns (list, optional): Column names for the input state.
        pred_columns (list, optional): Column names for the predicted state.
    """
    print("==== Flight State Prediction ====\n")
    print("Input Sequence (Last Timestep):")
    if input_columns is not None:
        last_input = input_seq[-1]
        for name, value in zip(input_columns, last_input):
            print(f"  {name:15s}: {value:.2f}")
    else:
        print(input_seq[-1])
    
    print("\nPredicted Next State:")
    if pred_columns is not None:
        pretty_print_state(prediction, pred_columns)
    else:
        print(prediction)

state_columns = ["lon", "lat", "heading1", "velocity", "vertrate", "heading2", "geoaltitude"]


In [41]:

# ---- Generate a Prediction ----
model.eval()
with torch.no_grad():
    # Get a sample input sequence (take the first sample from the test set)
    sample_input = X_test_t[0].unsqueeze(0)  # shape: (1, 17, 7)
    prediction = model(sample_input)         # shape: (1, 7)

# Remove the batch dimension and convert predictions (and input) to NumPy arrays.
prediction = prediction.squeeze(0).cpu().numpy()
sample_input_np = X_test_t[0].cpu().numpy()

# Print out the last timestep from the input and the predicted next state.
print_prediction(sample_input_np, prediction, input_columns=state_columns, pred_columns=state_columns)

==== Flight State Prediction ====

Input Sequence (Last Timestep):
  lon            : -2.61
  lat            : 53.54
  heading1       : 270.26
  velocity       : 225.84
  vertrate       : 0.65
  heading2       : 270.26
  geoaltitude    : 10919.46

Predicted Next State:
lon            : -36.61
lat            : 42.05
heading1       : 286.57
velocity       : 244.74
vertrate       : -0.47
heading2       : 286.66
geoaltitude    : 10669.73
